In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import xgboost
%matplotlib inline

In [2]:
train_df=pd.read_csv("train.csv")
#train_df=pd.get_dummies(train_df,columns=["year"])
train_df=train_df.drop(["Total Bags","type"],axis=1)
train_df.head()

,id,Total Volume,4046,4225,4770,Small Bags,Large Bags,XLarge Bags,year,AveragePrice
0,12697,1620.45,102.47,788.49,0.00,720.00,9.49,0.00,2016,1.57
1,8052,6347964.79,3529761.56,1305030.08,18206.06,1296031.40,183062.26,15873.43,2017,0.74
2,10155,1313.43,97.10,905.10,0.00,121.72,189.51,0.00,2015,1.78
3,1320,2710710.15,531460.61,1328248.62,203932.87,597061.79,48069.52,1936.74,2015,1.15
4,13668,9220.95,3417.54,3561.51,0.00,580.64,1661.26,0.00,2016,2.13


In [3]:
y=train_df["AveragePrice"]
y1=train_df[(train_df["id"]<=9125)]
y1=y1["AveragePrice"]
y1=y1.reset_index(drop=True)
y2=train_df[(train_df["id"]>9125)]
y2=y2["AveragePrice"]
y2=y2.reset_index(drop=True)
#y=np.multiply(y,100)
y2.head()

0    1.57
1    1.78
2    2.13
3    1.59
4    1.90
Name: AveragePrice, dtype: float64

In [4]:
x=train_df.drop(["AveragePrice"],axis=1)

In [5]:
x1=x[(x['id']<=9125)]
x1=x1.reset_index(drop=True)
x1.head()

,id,Total Volume,4046,4225,4770,Small Bags,Large Bags,XLarge Bags,year
0,8052,6347964.79,3529761.56,1305030.08,18206.06,1296031.40,183062.26,15873.43,2017
1,1320,2710710.15,531460.61,1328248.62,203932.87,597061.79,48069.52,1936.74,2015
2,689,257472.31,88573.56,43702.48,78776.60,33816.62,1081.83,11521.22,2015
3,2607,228585.31,135741.88,33351.68,40.46,40754.56,18696.73,0.00,2015
4,3541,127392.78,1165.74,82541.69,2576.45,39610.98,1472.27,25.65,2016


In [6]:
x2=x[(x['id']>9125)]
x2=x2.reset_index(drop=True)
x2.head()

,id,Total Volume,4046,4225,4770,Small Bags,Large Bags,XLarge Bags,year
0,12697,1620.45,102.47,788.49,0.0,720.00,9.49,0.0,2016
1,10155,1313.43,97.10,905.10,0.0,121.72,189.51,0.0,2015
2,13668,9220.95,3417.54,3561.51,0.0,580.64,1661.26,0.0,2016
3,10886,10924.58,8585.87,1454.55,0.0,866.67,17.49,0.0,2015
4,9156,1752.14,82.30,105.49,0.0,1564.35,0.00,0.0,2015


In [7]:
test_df=pd.read_csv("test.csv")
#test_df=pd.get_dummies(test_df,columns=["year"])
test_df.head()

,id,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year
0,11454,4156.05,610.35,1876.40,528.23,1141.07,1041.50,99.57,0.00,0,2015
1,13402,31716.46,3786.22,10296.42,600.95,17032.87,16887.13,145.74,0.00,0,2016
2,14460,5259.20,2152.85,2715.23,3.51,387.61,304.62,82.99,0.00,0,2016
3,6660,205429.57,13018.13,79071.99,1773.31,111566.14,46322.29,65171.80,72.05,1,2017
4,13707,7864.22,873.29,7.04,0.00,6983.89,6548.00,435.89,0.00,0,2016


In [8]:
FOODID=pd.read_csv("test.csv", usecols = ['id'])
FOODID.head()

,id
0,11454
1,13402
2,14460
3,6660
4,13707


In [9]:
test_df=test_df.drop(["Total Bags","type"],axis=1)
test_df.head()

,id,Total Volume,4046,4225,4770,Small Bags,Large Bags,XLarge Bags,year
0,11454,4156.05,610.35,1876.40,528.23,1041.50,99.57,0.00,2015
1,13402,31716.46,3786.22,10296.42,600.95,16887.13,145.74,0.00,2016
2,14460,5259.20,2152.85,2715.23,3.51,304.62,82.99,0.00,2016
3,6660,205429.57,13018.13,79071.99,1773.31,46322.29,65171.80,72.05,2017
4,13707,7864.22,873.29,7.04,0.00,6548.00,435.89,0.00,2016


In [10]:
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
kf = KFold(n_splits=5,shuffle=False,random_state=12)
kf.get_n_splits(x)
print(kf)

KFold(n_splits=5, random_state=12, shuffle=False)


In [11]:
        predicted_y = []
        expected_y = []
        for train_index, test_index in kf.split(x):
            #print("TRAIN:", train_index, "\nTEST:", test_index)
            x_train, x_test = x.loc[train_index], x.loc[test_index]
            y_train, y_test = y[train_index], y[test_index]
            model =xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.1,min_child_weight=0.25, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=10)
            model.fit(x_train,y_train)
            predicted_y.extend(model.predict(x_test))
            expected_y.extend(y_test)
        mse =mean_squared_error(expected_y,predicted_y)
        print(mse)

0.022856770125377703


In [12]:
kf = KFold(n_splits=5,shuffle=False,random_state=12)
kf.get_n_splits(x1)
print(kf)

KFold(n_splits=5, random_state=12, shuffle=False)


In [13]:
        predicted_y = []
        expected_y = []
        for train_index, test_index in kf.split(x1):
            #print("TRAIN:", train_index, "\nTEST:", test_index)
            x_train, x_test = x1.loc[train_index], x1.loc[test_index]
            y_train, y_test = y1[train_index], y1[test_index]
            model1 =xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.1,min_child_weight=0.25, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=10)
            model1.fit(x_train,y_train)
            predicted_y.extend(model1.predict(x_test))
            expected_y.extend(y_test)
        mse =mean_squared_error(expected_y,predicted_y)
        print(mse)    

0.013913967340165758


In [14]:
model1.get_booster().get_score(importance_type='weight')

{'4046': 10568,
 'id': 20889,
 'Large Bags': 9706,
 '4770': 10761,
 '4225': 9942,
 'Small Bags': 9482,
 'Total Volume': 12475,
 'XLarge Bags': 6498}

In [15]:
kf = KFold(n_splits=5,shuffle=False,random_state=12)
kf.get_n_splits(x2)
print(kf)

KFold(n_splits=5, random_state=12, shuffle=False)


In [16]:
        predicted_y = []
        expected_y = []
        for train_index, test_index in kf.split(x2):
            #print("TRAIN:", train_index, "\nTEST:", test_index)
            x_train, x_test = x2.loc[train_index], x2.loc[test_index]
            y_train, y_test = y2[train_index], y2[test_index]
            model2 =xgboost.XGBRegressor(n_estimators=2000, learning_rate=0.1,min_child_weight=0.25, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=8)
            model2.fit(x_train,y_train)
            predicted_y.extend(model2.predict(x_test))
            expected_y.extend(y_test)
        mse =mean_squared_error(expected_y,predicted_y)
        print(mse)

0.030534024443139657


In [17]:
model2.get_booster().get_score(importance_type='weight')

{'Large Bags': 10622,
 '4225': 14814,
 '4046': 16251,
 'Small Bags': 13468,
 'id': 27788,
 'Total Volume': 17497,
 '4770': 5489,
 'XLarge Bags': 231}

In [18]:
print(expected_y)
print(len(expected_y))

[1.57, 1.78, 2.13, 1.59, 1.9, 1.87, 1.61, 1.21, 1.84, 1.47, 1.54, 1.97, 1.85, 1.15, 1.65, 1.9, 2.0, 1.76, 1.09, 1.39, 1.78, 1.67, 2.14, 2.11, 1.86, 2.31, 1.81, 2.02, 1.01, 1.47, 1.76, 1.93, 1.88, 1.2, 1.73, 2.05, 1.81, 1.68, 1.62, 0.9, 1.68, 1.76, 1.91, 1.45, 1.69, 2.2, 1.54, 1.97, 1.86, 1.82, 1.06, 1.31, 1.79, 1.69, 1.41, 1.15, 1.58, 1.64, 2.02, 1.28, 1.31, 1.05, 1.68, 1.33, 1.62, 1.75, 0.87, 1.72, 1.79, 1.47, 1.61, 0.86, 2.35, 1.14, 1.63, 1.81, 1.68, 1.54, 1.48, 1.61, 2.21, 0.93, 1.72, 1.69, 1.92, 1.64, 1.66, 1.88, 1.52, 0.99, 1.48, 1.52, 1.16, 1.34, 1.81, 1.48, 1.41, 1.72, 2.11, 1.71, 1.69, 1.44, 1.53, 1.36, 1.42, 1.07, 1.87, 1.64, 1.44, 1.48, 1.79, 1.8, 1.66, 2.48, 1.67, 1.6, 1.26, 1.55, 1.37, 1.03, 1.42, 1.79, 1.43, 1.35, 2.54, 2.17, 1.27, 1.48, 1.9, 2.34, 1.47, 1.45, 1.55, 1.49, 1.82, 1.53, 1.72, 1.33, 2.07, 1.45, 2.35, 2.0, 1.8, 1.43, 1.8, 1.87, 1.79, 1.61, 1.66, 1.94, 1.24, 1.43, 1.86, 1.44, 1.64, 1.88, 1.74, 1.51, 1.32, 1.85, 1.69, 1.63, 1.45, 1.45, 1.6, 1.7, 1.92, 1.36, 1.36,

In [19]:
print(predicted_y)
print(len(predicted_y))

[1.5541794, 1.7773111, 2.145679, 1.5520241, 1.8335441, 1.7759689, 1.6166674, 1.5137122, 1.7714331, 1.3720348, 1.6610521, 1.7457781, 1.6574591, 1.2751403, 1.559593, 1.8267195, 2.1135302, 1.4871824, 1.3616352, 1.5261341, 1.4835742, 1.7401571, 1.7003851, 1.9260726, 1.7661209, 2.0824666, 1.7736689, 1.9356834, 1.1278785, 1.5114353, 1.6826851, 1.9261371, 1.7203244, 1.207156, 1.8946176, 1.8319521, 1.7351385, 1.6820703, 1.7468046, 0.9963021, 1.7118527, 1.720124, 1.7509484, 1.452333, 1.6954149, 1.4809604, 1.5091755, 2.1177204, 1.7378418, 1.432411, 1.1318451, 1.7041689, 1.9633173, 1.6681559, 1.4133451, 1.2856143, 1.7664152, 1.8210969, 1.945031, 1.5276275, 1.3314836, 1.2254095, 1.8930765, 1.6885142, 1.5353509, 1.6504128, 0.8858602, 1.6777939, 1.6888217, 1.3975494, 1.9106901, 0.73134637, 2.252131, 1.1265132, 1.7192678, 2.0486226, 1.6662823, 1.6331993, 1.4710962, 1.5491025, 2.0091777, 0.69322014, 1.8748443, 1.7307913, 1.9369197, 1.8061512, 2.0009396, 1.7092111, 1.5310901, 1.0213169, 1.3633449, 1.51

In [20]:
predicted_price=[]
for i in range(len(test_df)):
    xx=test_df[i:i+1]
    #xx=xx.reshape(1,15)
    print("{0} {1} {2}".format(model.predict(xx),model1.predict(xx),model2.predict(xx)))
    if test_df.iloc[i]['id']<=9125:
        #print(1111111)
        predicted_price.extend((model1.predict(xx)*0.75+model.predict(xx)*0.25))
    else :
        #print(2222222)
        predicted_price.extend((model2.predict(xx)+model.predict(xx))/2)
print(predicted_price)
print(len(predicted_price))

[1.9204676] [1.3742375] [1.9020246]
[1.9842199] [1.3291683] [2.0427089]
[2.1386256] [1.2312754] [2.1875305]
[1.0993719] [1.0943036] [1.3872719]
[1.5027204] [1.2294798] [1.5307363]
[1.295471] [1.2340181] [1.262255]
[1.3918164] [1.3588061] [1.406004]
[1.5884619] [1.1655169] [1.4918666]
[1.4091033] [1.4007852] [1.3296349]
[1.5671035] [1.2425745] [1.5698636]
[1.5295199] [1.2077694] [1.5172503]
[1.9383335] [1.2234075] [1.992924]
[1.6699948] [1.3647983] [1.7521812]
[1.2299411] [1.1927743] [1.262181]
[0.99628794] [0.96591246] [1.3989422]
[1.9010826] [1.2551082] [1.8456588]
[1.5718575] [1.1747427] [1.6215363]
[1.3386774] [1.2469563] [1.3592389]
[0.97956204] [0.9477234] [1.32528]
[1.9304601] [1.2784035] [1.94676]
[0.9408209] [0.8834789] [1.2369142]
[1.4906747] [1.540868] [1.2351363]
[1.3807769] [1.2478232] [1.4015201]
[1.1136436] [1.0990355] [1.1182766]
[1.5816966] [1.3700955] [1.4865744]
[2.1573944] [1.2515785] [2.1081085]
[1.7064046] [1.1808493] [1.7211269]
[1.7137544] [1.2373843] [1.6613967]

[1.3979006] [1.2683651] [1.341707]
[2.2623115] [1.2361274] [2.256518]
[1.084018] [1.1011434] [1.3343828]
[1.6377342] [1.2150482] [1.6552776]
[2.090743] [1.2314925] [2.0295644]
[1.1541159] [1.2278756] [1.1899118]
[1.1732183] [1.3146386] [1.3772538]
[1.6311107] [1.2512112] [1.5595617]
[1.0442767] [1.1086967] [1.2352738]
[1.2896516] [1.2468531] [1.3439093]
[2.0914779] [1.1845934] [2.0885432]
[1.9687678] [1.2496351] [2.055772]
[1.0652492] [1.061167] [1.2802501]
[1.558431] [1.2425745] [1.5718479]
[1.0530035] [1.0436685] [1.325858]
[1.4235296] [1.4267604] [1.5094128]
[2.0946174] [1.2439594] [2.1531687]
[1.1785378] [1.3822811] [1.2833323]
[1.5531095] [1.2296045] [1.5623108]
[1.4630821] [1.5467526] [1.2182668]
[1.2158084] [1.2603223] [1.358409]
[1.8787712] [1.2430091] [1.8640432]
[1.5525486] [1.2398083] [1.5267465]
[1.7435839] [1.2455978] [1.6940993]
[0.87567747] [0.88887775] [1.1452476]
[2.464836] [1.2299683] [2.4187148]
[0.8223639] [0.7982683] [1.3662723]
[1.4886141] [1.2440293] [1.4119413]


[1.7609206] [1.2448047] [1.7215707]
[1.3035734] [1.3125582] [1.4881377]
[1.154416] [1.1688454] [1.3225031]
[1.1126982] [1.1006589] [1.2332809]
[0.9245051] [0.8974582] [1.2105235]
[0.81104094] [0.8380274] [1.3318162]
[1.4951308] [1.5580145] [1.4876633]
[1.4934406] [1.4656942] [1.2909346]
[1.3579211] [1.3539004] [1.36356]
[1.649997] [1.0835702] [1.5697685]
[1.0509481] [1.0322502] [1.401442]
[1.7414423] [1.2425745] [1.7154863]
[1.3211385] [1.3160425] [1.5469096]
[1.9541786] [1.2525132] [1.950987]
[1.8924179] [1.2102661] [1.9605898]
[1.2471355] [1.2116046] [1.4540117]
[0.97191083] [0.98156285] [1.1816471]
[1.7224739] [1.2543838] [1.7843782]
[1.0024922] [1.0182881] [1.2148703]
[2.0507345] [1.3814499] [2.0687444]
[1.5181024] [1.1619292] [1.470725]
[1.1458902] [1.1328382] [1.2102579]
[1.4805803] [1.5869793] [1.4407175]
[1.3457756] [1.1946557] [1.4154022]
[1.7837818] [1.2504529] [1.8166804]
[0.9682281] [1.0390553] [1.2460065]
[1.6966984] [1.2564926] [1.6945306]
[1.1062784] [1.2494528] [0.99780

[1.5406553] [1.4881402] [1.4443153]
[1.6878483] [1.2326126] [1.7159439]
[2.0478654] [1.2115316] [2.0357199]
[1.1096768] [1.201852] [1.280014]
[0.99209046] [0.96088177] [1.3091269]
[1.0046754] [0.9860079] [1.2168515]
[1.0488358] [1.0424893] [1.3321261]
[1.5159001] [1.3289126] [1.57733]
[0.86636543] [0.8913042] [1.3200703]
[1.1502936] [1.1290486] [1.2135804]
[1.8171332] [1.2231213] [1.8234458]
[1.1195552] [1.1603806] [0.9391365]
[1.6816845] [1.2418063] [1.6918352]
[1.8279941] [1.152074] [1.8442196]
[1.6485523] [1.2326584] [1.5997266]
[1.0780602] [1.0607299] [1.2858973]
[1.6145074] [1.6060479] [1.2780007]
[1.2228097] [1.225894] [1.3821058]
[1.5730689] [1.2302768] [1.6226525]
[1.4525096] [1.2513175] [1.5343748]
[2.0691814] [1.2341704] [1.9772277]
[1.2277037] [1.1049815] [1.375258]
[1.3969511] [1.0892546] [1.4576138]
[1.2194743] [1.2500236] [1.2297742]
[1.6540978] [1.2425745] [1.6925992]
[1.7331384] [1.2512112] [1.7918235]
[1.759795] [1.23091] [1.6710616]
[1.8982222] [1.2500236] [1.9950323]

[1.1340497] [1.0859096] [1.4710629]
[1.2341313] [1.2080052] [1.5148834]
[1.092354] [1.2361054] [1.2364879]
[1.3497245] [1.2513312] [1.3421271]
[1.0384972] [1.0031564] [1.3951281]
[1.9404018] [1.3718402] [1.9850687]
[1.5923274] [1.2352034] [1.6158364]
[1.7754396] [1.2270719] [1.8048562]
[0.7708703] [0.83184147] [1.2716699]
[1.3681986] [1.2193043] [1.3773432]
[1.4206009] [1.1865339] [1.5684221]
[1.7482482] [1.2168634] [1.7658298]
[1.2317023] [1.228431] [1.2996709]
[0.7882557] [0.8072601] [1.3752854]
[1.2051907] [1.1649717] [1.2681375]
[1.5796041] [1.6996927] [1.2736228]
[0.9638562] [0.98217404] [1.2688861]
[1.2351143] [1.2356404] [1.3939354]
[1.005119] [0.94531226] [1.1632643]
[1.3879399] [1.3716104] [1.2078531]
[1.6808094] [1.2520635] [1.6949464]
[1.1454438] [1.1800814] [1.3483807]
[1.5920578] [1.2157336] [1.6696765]
[1.1766057] [1.1909018] [1.1098456]
[1.455975] [1.4502602] [0.9992982]
[0.8038806] [0.7929317] [1.340738]
[1.3792033] [1.2485087] [1.4276931]
[1.4379989] [1.487827] [1.1664

...

[0.9786911] [1.010208] [1.3116333]
[1.6302671] [1.2359872] [1.6670328]
[1.6406777] [1.1680219] [1.5912136]
[0.8371778] [0.8634382] [1.3723943]
[1.6882504] [1.1242647] [1.7693796]
[1.5197506] [1.235883] [1.5333551]
[0.96638477] [0.9906845] [1.4189105]
[1.1809757] [1.1114011] [1.3624566]
[1.1543131] [1.153947] [1.4729552]
[1.2443666] [1.2219818] [1.3135679]
[1.3028358] [1.2493566] [1.2366083]
[1.235316] [1.3324342] [1.2988682]
[1.7009431] [1.2235688] [1.6631262]
[1.0727984] [0.95147765] [1.2897904]
[1.1021919] [1.098659] [1.2767358]
[1.7428616] [1.3568048] [1.6404694]
[1.4603047] [1.1301262] [1.561954]
[1.2992494] [1.3056648] [1.6220053]
[1.7574375] [1.2409716] [1.7068974]
[0.77748907] [0.8222296] [1.2802501]
[1.3334813] [1.330646] [1.1741692]
[0.9615994] [0.97695935] [1.3065374]
[1.3874052] [1.3464501] [1.2998247]
[1.5261568] [1.2338485] [1.5363652]
[1.1003625] [1.0572813] [1.4273953]
[1.3646581] [1.1346111] [1.3722904]
[1.1701387] [1.1615763] [1.4368606]
[1.1152322] [1.0665979] [1.2836

[1.0057868] [0.9386431] [1.4616047]
[1.4065182] [1.4733799] [1.3838475]
[1.9340692] [1.306474] [1.9263837]
[1.4035801] [1.3545461] [1.418891]
[1.1443157] [1.2087255] [1.4505458]
[1.6602342] [1.2522433] [1.6020321]
[1.3407418] [1.2163749] [1.3147635]
[1.3305147] [1.4043262] [1.2293861]
[0.9907297] [0.90267134] [1.3417499]
[0.95918345] [0.94226575] [1.2895544]
[1.4091043] [1.3867753] [1.3408567]
[1.6276051] [1.2287123] [1.533544]
[1.091979] [1.0415308] [1.1904149]
[1.2044125] [1.2332119] [1.0652018]
[1.6117117] [1.2448047] [1.4561325]
[1.5779208] [1.2561977] [1.5243708]
[1.8778297] [1.3745117] [1.812]
[0.96148497] [1.0197965] [1.200146]
[1.9079194] [1.2398648] [1.9196212]
[0.981003] [0.97830856] [1.2802501]
[1.4690673] [1.2505105] [1.5254561]
[1.5118297] [1.1369158] [1.5179176]
[1.3173544] [1.0875868] [1.332873]
[1.3423874] [1.3370017] [1.3185116]
[1.3983748] [1.2517202] [1.4768915]
[1.4425395] [1.315572] [1.4320297]
[1.5060189] [1.2541156] [1.5207855]
[1.0046124] [1.022569] [1.1557102]


[0.96817267] [0.9651724] [1.3461275]
[1.5866547] [1.1991568] [1.6244372]
[0.88940954] [0.8831755] [1.3127692]
[1.4988084] [1.4016188] [1.3680364]
[1.6162926] [1.59936] [1.2320914]
[1.3107735] [1.319755] [1.2390249]
[2.0972986] [1.0933778] [2.0708792]
[1.2201109] [1.1548393] [1.0333519]
[2.0080066] [1.2496368] [2.0054007]
[1.1760434] [1.2459378] [1.3613468]
[2.3687272] [1.2431841] [2.3564699]
[0.9790218] [1.0056098] [1.2773542]
[0.8105452] [0.87700456] [1.2205582]
[1.234207] [1.2315888] [1.4665899]
[0.9824655] [0.9304805] [1.2341996]
[1.3509698] [1.402124] [1.2602326]
[1.0157964] [0.9937328] [1.139252]
[1.5439919] [1.2375762] [1.5056273]
[1.2110355] [1.2268809] [1.2906983]
[1.4642088] [1.3984709] [1.4302332]
[1.1893734] [1.2396281] [1.2996358]
[2.0369697] [1.2425745] [2.085606]
[1.1588597] [1.2120242] [1.4049613]
[1.2669224] [1.2332554] [1.1003294]
[1.4271739] [1.1947198] [1.419127]
[0.9755266] [1.0410337] [1.1686207]
[1.3836026] [1.4503582] [1.286735]
[0.5532517] [0.5514822] [1.2860405

[1.1339931] [1.1673105] [1.249423]
[0.96925783] [0.96829057] [1.285963]
[1.9162037] [1.4051688] [1.984825]
[1.0441827] [1.037928] [1.3074491]
[1.6227885] [1.6785152] [1.5350069]
[1.7850959] [1.1667582] [1.887417]
[1.754877] [1.3770118] [1.7884911]
[2.0092921] [1.328985] [2.009398]
[1.1010106] [1.1167842] [1.3087804]
[1.1781616] [1.1546738] [1.3456099]
[1.2678015] [1.2783246] [1.4967357]
[1.7261866] [1.2246153] [1.7396364]
[1.9499588] [1.2244117] [1.9056108]
[1.2131963] [1.1640422] [1.28654]
[1.1636902] [1.1671202] [1.1391447]
[1.3792944] [1.2396114] [1.4118905]
[1.6139458] [1.2532779] [1.7064558]
[1.0856414] [1.068109] [1.4864786]
[1.1560826] [1.2222512] [1.4014659]
[1.0669309] [1.1034808] [1.2802501]
[0.92167413] [0.9790147] [1.1251158]
[1.481612] [1.50319] [1.178679]
[1.3204908] [1.3131931] [1.4580197]
[0.93485034] [0.86881906] [1.1577528]
[1.441609] [1.4093156] [1.1156409]
[1.708026] [1.2119211] [1.7865276]
[1.2781718] [1.3149672] [1.3644766]
[1.4601221] [1.1543627] [1.2498562]
[2.1

[1.4890003] [1.5625383] [1.4512331]
[2.1431818] [1.2218642] [2.246809]
[1.5707569] [1.5234569] [1.3016646]
[1.919646] [1.2399225] [2.0079904]
[1.2410302] [1.2231019] [1.3741896]
[1.8486077] [1.228879] [1.8469942]
[1.3419142] [1.3110058] [1.5634768]
[1.3964739] [1.2424495] [1.3936512]
[1.5117978] [1.2474959] [1.3970623]
[1.5836372] [1.2067761] [1.6207174]
[1.8569796] [1.2302346] [1.8689836]
[1.0666339] [1.0788956] [1.3079621]
[1.5688899] [1.2485082] [1.626081]
[1.5742164] [1.5596232] [1.2463707]
[2.1113777] [1.2170944] [1.9793721]
[1.1466424] [1.1475263] [1.257688]
[1.0568328] [1.0479524] [1.3464522]
[1.2084932] [1.2231582] [1.4045142]
[1.741136] [1.2476342] [1.7974446]
[0.91544753] [0.937855] [1.2041624]
[0.95164084] [0.9444075] [1.1489272]
[1.4053345] [1.2478807] [1.3901699]
[1.6503181] [1.2425745] [1.7491441]
[0.90201986] [0.9146308] [1.1161342]
[1.420836] [1.2122153] [1.450096]
[0.9096532] [0.8563745] [1.1978838]
[2.3547955] [1.2431841] [2.3260489]
[1.8345424] [1.2101898] [1.8178486

In [21]:
FOODID.head()

,id
0,11454
1,13402
2,14460
3,6660
4,13707


In [22]:
final  = pd.concat([FOODID, pd.DataFrame(predicted_price)], axis=1)
final.head(20)

,id,0
0,11454,1.911246
1,13402,2.013464
2,14460,2.163078
3,6660,1.095571
4,13707,1.516728
5,4369,1.249381
6,1708,1.367059
7,17064,1.540164
8,4116,1.402865
9,14745,1.568484


In [23]:
final.to_csv("result_last.csv", index=False)